Forecasting blood supply is a serious and recurrent problem for blood collection managers.In this Project,I have  worked  with data collected from the donor database of Blood TransfusionService Center.  The dataset, obtained from the Machine Learning Repository, consists of a random sample of 748 donors. 


# **BloodTransfusion Project**

**Import Necessary Library**


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
transfusion = pd.read_csv('/content/transfusion.csv')
transfusion

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0
...,...,...,...,...,...
743,23,2,500,38,0
744,21,2,500,52,0
745,23,3,750,62,0
746,39,1,250,39,0


In [ ]:
transfusion.head()

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),whether he/she donated blood in March 2007
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1
3,2,20,5000,45,1
4,1,24,6000,77,0


In [ ]:
transfusion.shape

(748, 5)

In [ ]:
!ls

sample_data  transfusion.csv


In [ ]:
transfusion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 748 entries, 0 to 747
Data columns (total 5 columns):
 #   Column                                      Non-Null Count  Dtype
---  ------                                      --------------  -----
 0   Recency (months)                            748 non-null    int64
 1   Frequency (times)                           748 non-null    int64
 2   Monetary (c.c. blood)                       748 non-null    int64
 3   Time (months)                               748 non-null    int64
 4   whether he/she donated blood in March 2007  748 non-null    int64
dtypes: int64(5)
memory usage: 29.3 KB


In [ ]:
transfusion.rename(columns={'whether he/she donated blood in March 2007':'Target'},inplace=True)

In [ ]:
transfusion.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months),Target
0,2,50,12500,98,1
1,0,13,3250,28,1


In [ ]:
print(round(transfusion['Target'].value_counts(normalize=True)*100,3))

0    76.203
1    23.797
Name: Target, dtype: float64


In [ ]:
transfusion['Target'].value_counts()

0    570
1    178
Name: Target, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    transfusion.drop(columns='Target',axis=1),
    transfusion.Target,
    test_size=0.25,
    random_state=42,
    stratify=transfusion.Target
)

X_train.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
334,16,2,500,16
99,5,7,1750,26


In [ ]:
x_train.head(2)

,Recency (months),Frequency (times),Monetary (c.c. blood),Time (months)
97,4,9,2250,38
516,3,5,1250,12


In [ ]:
from tpot import TPOTClassifier
from sklearn.metrics import roc_auc_score

In [ ]:
!pip install tpot

In [ ]:

tpot = TPOTClassifier(
    generations=7,
    population_size=20,
    verbosity=2,
    scoring='roc_auc',
    random_state=42,
    disable_update_check=True,
    config_dict='TPOT light'
)
tpot.fit(X_train, y_train)

tpot_auc_score = roc_auc_score(y_test, tpot.predict_proba(X_test)[:, 1])
print(f'\nAUC score: {tpot_auc_score:.4f}')

print('\nBest pipeline steps:', end='\n')
for idx, (name, transform) in enumerate(tpot.fitted_pipeline_.steps, start=1):
    print(f'{idx}. {transform}')


Generation 1 - Current best internal CV score: 0.7422459184429089

Generation 2 - Current best internal CV score: 0.7422459184429089

Generation 3 - Current best internal CV score: 0.7422459184429089

Generation 4 - Current best internal CV score: 0.7422459184429089

Generation 5 - Current best internal CV score: 0.7456308339276876

Generation 6 - Current best internal CV score: 0.7464101394881147

Generation 7 - Current best internal CV score: 0.7464101394881147

Best pipeline: LogisticRegression(CombineDFs(Normalizer(StandardScaler(input_matrix), norm=l2), StandardScaler(input_matrix)), C=25.0, dual=False, penalty=l2)

AUC score: 0.7919

Best pipeline steps:
1. FeatureUnion(n_jobs=None,
             transformer_list=[('pipeline',
                                Pipeline(memory=None,
                                         steps=[('standardscaler',
                                                 StandardScaler(copy=True,
                                                             

In [ ]:
print(round(X_train.var(),3))

Recency (months)              66.929
Frequency (times)             33.830
Monetary (c.c. blood)    2114363.700
Time (months)                611.147
dtype: float64


In [ ]:
import numpy as np

X_train_normed, X_test_normed= X_train.copy(), X_test.copy()

col_to_normalize = 'Monetary (c.c. blood)'

for df_ in [X_train_normed, X_test_normed]:
    df_['monetary_log'] = np.log(df_[col_to_normalize])
    df_.drop(columns = col_to_normalize, inplace=True)

In [ ]:

print(round(X_train_normed.var(),3))

Recency (months)      66.929
Frequency (times)     33.830
Time (months)        611.147
monetary_log           0.837
dtype: float64


In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(
    solver='liblinear',
    random_state=42
)

logreg.fit(X_train_normed, y_train)

logreg_auc_score = roc_auc_score(y_test, logreg.predict_proba(X_test_normed)[:, 1])
print(f'\nAUC score: {logreg_auc_score:.4f}')


AUC score: 0.7890


In [ ]:
from operator import itemgetter

sorted(
    [('tpot', tpot_auc_score), ('logreg', logreg_auc_score)],
    key = itemgetter(1),
    reverse = True
)

[('tpot', 0.7918785759694851), ('logreg', 0.7890178003814368)]

#                                           Thank you !